<a href="https://colab.research.google.com/github/duper203/RAG_Techniques_with_upstage/blob/main/upstage/06_query_transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Query Transformations for Improved Retrieval in RAG Systems

## Overview

This code implements three query transformation techniques to enhance the retrieval process in Retrieval-Augmented Generation (RAG) systems:

1. Query Rewriting
2. Step-back Prompting
3. Sub-query Decomposition


Each technique aims to improve the relevance and comprehensiveness of retrieved information by modifying or expanding the original query.

## Method Details

1. Query Rewriting

  To make queries more specific and detailed, improving the likelihood of retrieving relevant information.
2. Step-back Prompting

  To generate broader, more general queries that can help retrieve relevant background information.
3. Sub-query Decomposition

  To break down complex queries into simpler sub-queries for more comprehensive information retrieval.

## Example Use Case

The code demonstrates each technique using the example query: "What are the impacts of climate change on the environment?"

* **Query Rewriting** expands this to include specific aspects like temperature changes and biodiversity.
* **Step-back Prompting** generalizes it to "What are the general effects of climate change?"
* **Sub-query Decomposition** breaks it down into questions about biodiversity, oceans, weather patterns, and terrestrial environments.


In [ ]:
! pip3 install -qU langchain-upstage langchain

In [2]:
import os
from google.colab import userdata

os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")

## 1 - Query Rewriting: Reformulating queries to improve retrieval.


In [13]:
from langchain_upstage import ChatUpstage
from langchain.prompts import PromptTemplate

re_write_llm = ChatUpstage(model="solar-pro")

# Create a prompt template for query rewriting
query_rewrite_template = """You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system.
Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information.

Original query: {original_query}

Rewritten query:"""

query_rewrite_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=query_rewrite_template
)

# Create an LLMChain for query rewriting
query_rewriter = query_rewrite_prompt | re_write_llm

def rewrite_query(original_query):
    """
    Rewrite the original query to improve retrieval.

    Args:
    original_query (str): The original user query

    Returns:
    str: The rewritten query
    """
    response = query_rewriter.invoke(original_query)
    return response.content

## Demonstrate on a use case

In [14]:
# example query over the understanding climate change dataset
original_query = "What are the impacts of climate change on the environment?"
rewritten_query = rewrite_query(original_query)
print("Original query:", original_query)
print("\nRewritten query:", rewritten_query)

Original query: What are the impacts of climate change on the environment?

Rewritten query: What are the specific environmental impacts of climate change, including its effects on global temperature, sea levels, biodiversity, and natural disasters?


## 2 - Step-back Prompting: Generating broader queries for better context retrieval.

In [15]:
step_back_llm = ChatUpstage(model="solar-pro")


# Create a prompt template for step-back prompting
step_back_template = """You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.

Original query: {original_query}

Step-back query:"""

step_back_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=step_back_template
)

# Create an LLMChain for step-back prompting
step_back_chain = step_back_prompt | step_back_llm

def generate_step_back_query(original_query):
    """
    Generate a step-back query to retrieve broader context.

    Args:
    original_query (str): The original user query

    Returns:
    str: The step-back query
    """
    response = step_back_chain.invoke(original_query)
    return response.content

## Demonstrate on a use case

In [16]:
# example query over the understanding climate change dataset
original_query = "What are the impacts of climate change on the environment?"
step_back_query = generate_step_back_query(original_query)
print("Original query:", original_query)
print("\nStep-back query:", step_back_query)

Original query: What are the impacts of climate change on the environment?

Step-back query: How does climate change affect the natural world?


## 3- Sub-query Decomposition: Breaking complex queries into simpler sub-queries.

In [18]:
sub_query_llm = ChatUpstage(model="solar-pro")

# Create a prompt template for sub-query decomposition
subquery_decomposition_template = """You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.

Original query: {original_query}

example: What are the impacts of climate change on the environment?

Sub-queries:
1. What are the impacts of climate change on biodiversity?
2. How does climate change affect the oceans?
3. What are the effects of climate change on agriculture?
4. What are the impacts of climate change on human health?"""


subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

# Create an LLMChain for sub-query decomposition
subquery_decomposer_chain = subquery_decomposition_prompt | sub_query_llm

def decompose_query(original_query: str):
    """
    Decompose the original query into simpler sub-queries.

    Args:
    original_query (str): The original complex query

    Returns:
    List[str]: A list of simpler sub-queries
    """
    response = subquery_decomposer_chain.invoke(original_query).content
    sub_queries = [q.strip() for q in response.split('\n') if q.strip() and not q.strip().startswith('Sub-queries:')]
    return sub_queries

## Demonstrate on a use case

In [19]:
# example query over the understanding climate change dataset
original_query = "What are the impacts of climate change on the environment?"
sub_queries = decompose_query(original_query)
print("\nSub-queries:")
for i, sub_query in enumerate(sub_queries, 1):
    print(sub_query)


Sub-queries:
1. What are the impacts of climate change on the global temperature?
2. How does climate change affect precipitation patterns and extreme weather events?
3. What are the consequences of climate change on sea level rise?
4. How does climate change impact natural habitats, such as forests, wetlands, and polar regions?
